Test a bunch of notebooks. Bioinfo notebooks are at ~/repos/bioinfo/test.ipynb

In [1]:
from k1lib.imports import *
from k1lib.bioinfo.cli import *
from IPython.display import clear_output

In [2]:
def execute(nb:str):
    print(nb)
    k1lib.executeNb(nb, globals())
    clear_output()

In [3]:
base = "/home/kelvin/repos/labs/k1lib/k1lib"
tests = []
for i in range(5): tests += glob.glob(f"{base}{'/*' * i}.ipynb")
tests.remove(f"{base}/test.ipynb")
for i, test in enumerate(tests):
    print(i); execute(test)

In [4]:
tut = "/home/kelvin/repos/labs/k1lib/docs/tutorials"
ls(tut) | grep(".ipynb$") | headOut()

/home/kelvin/repos/labs/k1lib/docs/tutorials/basics.ipynb
/home/kelvin/repos/labs/k1lib/docs/tutorials/mp.ipynb
/home/kelvin/repos/labs/k1lib/docs/tutorials/schedule.ipynb
/home/kelvin/repos/labs/k1lib/docs/tutorials/mo.ipynb
/home/kelvin/repos/labs/k1lib/docs/tutorials/osic.ipynb
/home/kelvin/repos/labs/k1lib/docs/tutorials/cli.ipynb
/home/kelvin/repos/labs/k1lib/docs/tutorials/covid.ipynb


In [5]:
execute(f"{tut}/covid.ipynb")
execute(f"{tut}/cli.ipynb")
execute(f"{tut}/schedule.ipynb")
execute(f"{tut}/mo.ipynb")
execute(f"{tut}/osic.ipynb")
execute(f"{tut}/mp.ipynb")
execute(f"{tut}/basics.ipynb")

In [8]:
execute("/home/kelvin/repos/labs/projects/char rnn/1) init/1) init.ipynb")

In [9]:
execute("/home/kelvin/repos/labs/projects/function/0) integrity/main.ipynb")

In [10]:
k1lib.beep()